In [1]:
%pylab inline
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sys

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
df_als = pd.read_csv("/data/home/taras.svirsky/lab12/res/res_als_r25_ni15.csv")
print df_als.shape
df_als.head(2)

(10525761, 3)


,userId,movieId,rating
0,1,1414,3.920710
1,1,2346,3.861442


In [3]:
df_i2i = pd.read_csv("/data/home/taras.svirsky/lab12/res/full_test_bp_i2i_top100.csv")
print df_i2i.shape
df_i2i.head(2)

(10531564, 4)


,userId,movieId,bp_predicted_rating,predicted_i2i_rating
0,28,2111,3.148587,3.248510
1,30,22037,3.146315,4.669889


In [6]:
df_als = df_als.rename(columns={"rating":"als_rating"})
df_i2i = df_i2i.rename(columns={"predicted_i2i_rating":"i2i_rating"})

In [7]:
df = df_i2i.merge(df_als, on=["userId","movieId"], how='left')

In [8]:
idx = df["als_rating"].isnull()
df.loc[idx, "als_rating"] = df.loc[idx, "bp_predicted_rating"]

In [9]:
del df_als
del df_i2i

In [10]:
df_cat = pd.read_csv("/data/home/taras.svirsky/lab12/res/test_subm_rating_counts.csv")

In [11]:
df = df.merge(df_cat, on=["userId","movieId"], how='left')

In [12]:
df['n_user_ratings'] = df['n_user_ratings'].fillna(0)
df['n_item_ratings'] = df['n_item_ratings'].fillna(0)

In [13]:
sum(df.isnull())

userId                 0
movieId                0
bp_predicted_rating    0
i2i_rating             0
als_rating             0
n_user_ratings         0
n_item_ratings         0
dtype: int64

In [16]:
assert df.shape[0] == 10531564

In [ ]:
del df_cat

# combine all

In [18]:
def cut_dot5to5(x):
    if x < .5:
        return .5
    if x > 5:
        return 5.0
    return x

In [19]:
idx_big = (df.n_user_ratings > 50) & (df.n_item_ratings>100)
idx_med = (df.n_user_ratings > 5) & (df.n_user_ratings <= 50) & (df.n_item_ratings>100)
idx_small = (~idx_big) & (~idx_med)

print sum(idx_big), sum(idx_med), sum(idx_small)
n = sum(idx_big) + sum(idx_med) + sum(idx_small)
assert n == 10531564
print 100.*sum(idx_big)/n, 100.*sum(idx_med)/n, 100.*sum(idx_small)/n


7719476 2422037 390051
73.2984768454 22.9978852144 3.7036379402


In [20]:
# v 351
#df["rating"] = np.where((df.n_item_ratings>100) &  (df.n_user_ratings > 5),
#                        0.88 * df["als_rating"] + 0.15 * df["i2i_rating"],
#                        0.38 * df["als_rating"] + 0.62 * df["i2i_rating"],
#                       )

In [21]:
# v 8010
df["rating"] = df["bp_predicted_rating"]
df['i2i_x_als'] = df['als_rating']*df['i2i_rating']

df.loc[idx_big,"rating"] =  -0.35258 * df.loc[idx_big, "bp_predicted_rating"] \
                          + 1.23212959 * df.loc[idx_big, "als_rating"] \
                          + 0.15828964 * df.loc[idx_big, "i2i_rating"]

df.loc[idx_med,"rating"] =  0.933 * df.loc[idx_med, "als_rating"] \
                          + 0.211 * df.loc[idx_med, "i2i_rating"] \
                          -0.0292 * df.loc[idx_med, "i2i_x_als"]


df.loc[idx_small,"rating"] =  0.483 * df.loc[idx_small, "als_rating"] \
                             + 0.65 * df.loc[idx_small, "i2i_rating"] \
                             -0.034 * df.loc[idx_small, "i2i_x_als"]



In [22]:
res = df[["userId", "movieId", "rating"]].sort_values(by=["userId", "movieId"])

In [23]:
res.rating.describe()

count    1.053156e+07
mean     3.530147e+00
std      6.726079e-01
min     -6.802913e-02
25%      3.138098e+00
50%      3.603708e+00
75%      4.002132e+00
max      5.771252e+00
Name: rating, dtype: float64

In [24]:
res['rating'] = res['rating'].map(cut_dot5to5)

In [26]:
res.rating.describe()

count    1.053156e+07
mean     3.529896e+00
std      6.720087e-01
min      5.000000e-01
25%      3.138098e+00
50%      3.603708e+00
75%      4.002132e+00
max      5.000000e+00
Name: rating, dtype: float64

In [25]:
res.to_csv("/data/home/taras.svirsky/lab12/res/full_test_v8010.csv", index=False)